Outline:

* Async/Await
* What is aiohttp
* Basic App
* Routing
* JSON/Form/etc
* Running It
* Deploying It
* Middlewares
* Start Up/Clean Up
* Websockets
* Extensions/Other Packages
* Questions

# aiohttp: An Introduction

Alec Nikolas Reiter
PyATL Oct. 2017

# async/await

* Successor to `yield from`
* Python 3.5
* `coroutines`
* event loop

# aiohttp

* HTTP Toolkit
    * `aiohttp.client`
    * `aiohttp.web`
* Minimal

# aiohttp.web

* server
* high level application
* "low level" server

```python
import asyncio
from aiohttp.web import Application, json_response, run_app


async def hello(request):
    return json_response({'message': 'hello'})

app = Application()
app.router.add_get('/', hello)
run_app(app, port=5000, loop=asyncio.get_event_loop())
```

## Routing

* Django-ish and Flask-ish
* Functions and Classes
* Subapplications

### URL Variables

```python
async def name(request):
    name = request.match_info['name']
    # ...
    
app.router.add_get('/{name:\w+}', name)
```

### Function Route Handler

```python
async def hello_name(request):
    name = request.match_info['name']
    return json_response({'message': f'hello {name}'})

app = Application()
app.router.add_get('/{name:\w+}', hello_name)
```

### Class Route Handler

```python
from aiohttp.web import View

class Hello(View):
    async def get(self):
        name = self.request.match_info['name']
        return json_response({'message': f'hello {name}'})
    
app = Application()
app.router.add_route('*', '/{name:\w+}', Hello)
```

### Composing Apps Together

```python
parent_app = Application()
parent_app.add_subapp('/name/', app)
```

## Request Data

* request.json()
* request.post()
* request.multipart()

# Running It

### run_app

* `aiohttp.web.run_app`
* Development Server
* Accepts Event Loop
* Quick and Easy

### CLI

```bash
python -m aiohttp.web -H localhost -P 8080 package.module:app_factory
```
* Development Server
* Needs factory function
* Unknown args go to factory

```python
def app_factory(argv):   
    app = Application()
    
    if '--hello-name' in argv:
        app.router.add_get('/{name:\w+}', hello_name)
        
    return app
```

## Deploying It

* Gunicorn
* aiohttp prepackages workers
    * `asyncio.event_loop` based
    * `uvloop` based
* uWSGI has experimental asyncio support

# More features

## Middleware

* Coroutine factory
* Return request handler
* *Run every request*

```python
from aiohttp.web import Application

async def request_counter(app, handler):
    async def handle(request):
        app['counter'] += 1
        return await handler(request)
    return handle

app = Application(middlewares=[request_counter])
app['counter'] = 0
```

## Signals

* Start up dependencies
* Redis, Message Queue, WSDL, etc
* Accept Coroutines
* Runs *in order*

## Signals Available

* `on_startup`
* `on_shutdown`
* `on_cleanup`

```python
from aiohttp.web import Application
from aiohttp.client import ClientSession

async def attach_http_client(app):
    app['http_client'] = ClientSession(loop=loop)
    

async def close_http_client(app):
    app['http_client'].close()
    
app = Application()
app.on_startup.append(attach_http_client)
app.on_shutdown.append(close_http_client)
```

## Signals: Buyer Beware

* You're responsible
* aiojobs helper lib

## Websockets

* Socket between client and server
* Allow push from both
* Mostly supported

```python
from aiohttp.web import WebSocketResponse

async def ws_handler(request):
    socket = WebSocketResponse(request)
    await socket.prepare()
    
    async for message in socket:
        # handle message
        
    return socket
```

## Websockets

* Only one reader allowed
* Multiple pushers
* Cleaned up when client disconnects
* Need cleanup at shutdown

## Other packages

* aiopg/aiomysql
* aioredis/aiomemcache
* aiohttp specific:
    * jinja2
    * debugtoolbar
    * cors
    * security
    * more

# Questions?